In [3]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "round summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 3. TURN: 1 


You call out a bid of $200, throwing your hat into the ring with a confident shout. The other bidders pause for a moment, sizing you up before the action resumes. A man in a worn-out jacket raises the bid to $250.

possible actions:
1. Raise the bid to $300.
2. Hold your bid and wait to see how the auction progresses.
3. Drop out of the bidding.

I select option 3. TURN: 2 


You decide to step back, letting the other bidders fight it out. The auctioneer's voice fills the air as the price climbs higher and higher. Finally, the bidding war ends at $350, won by a tall, slender woman with an air of confidence. As she signs the paperwork, the crowd begins to disperse, leaving you to ponder your next move.

possible actions:
1. Approach the winner to see if you can take a look inside the storage with her.
2. Talk to the auctioneer about upcoming auctions.
3. Leave the auction and head to a nearby diner to rethink your strategy.

I select option 1. TURN: 3 


You approach the tall, slender woman who won the auction, introducing yourself with a polite nod. She seems intrigued by your interest and, after a brief conversation, agrees to let you take a quick look inside the storage with her. As the roller shutter opens fully, the dim light reveals an assortment of items: old books, furniture that has seen better days, and what appears to be a collection of old, strange-looking artifacts.

possible actions:
1. Examine the strange-looking artifacts more closely.
2. Offer to help the woman move some of the items to get a better look at what's in the back.
3. Ask the woman about her interest in this particular storage unit.

I select option 1. TURN: 4 


skill: spot_hidden
difficulty: medium

I am making a skill check using SPOT_HIDDEN against a difficulty of MEDIUM.
And I rolled a 77 for a result of FAILURE. TURN: 5 


As you lean in to examine the strange-looking artifacts more closely, your eyes struggle to make out the finer details in the dim light of the storage unit. The shapes and symbols on the artifacts blur together, making it impossible for you to discern their significance or value. Frustrated by your inability to uncover anything useful, you step back, feeling a pang of disappointment.

The woman, noticing your interest in the artifacts, offers a sympathetic smile before turning her attention back to the task of assessing her newly acquired treasures.

round summary: Kyle Jackson, after bidding unsuccessfully at the storage auction, managed to gain a brief look inside the winning storage unit. Despite his attempt to examine some intriguing artifacts, his effort was hampered by a failure to spot any hidden details, leaving him with no choice but to step back an